# Loan Default Predictor Model

##  Data Exploration and Pre-processing:

### Import libraries and datasets:

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing 
import seaborn as sns
import xgboost as xgb # type: ignore
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split,GridSearchCV
import joblib
from sklearn.metrics import f1_score
%matplotlib inline

In [13]:
merchantDetails = pd.DataFrame({

})

loanScheduleDetails = pd.DataFrame({
 
})

loanLedgerDetails = pd.DataFrame({
 
})

transactionDetails = pd.DataFrame({

})

### Formatting

In [ ]:
# Rename columns with same name

merchantDetails.rename(columns={'id': 'merchant_code', 'created_at': 'merchant_created_at', 'updated_at':'merchant_updated_at'}, inplace=True)

loanScheduleDetails.rename(columns={'id': 'loan_schedule_id', 'created_at': 'loan_schedule_created_at', 'updated_at':'loan_schedule_updated_at', 'paid_date':'loan_schedule_paid_date'}, inplace=True)

loanLedgerDetails.rename(columns={'id': 'loan_id', 'created_at': 'loan_created_at', 'updated_at':'loan_updated_at', 'deleted_at':'loan_deleted_at', 'transaction_date':'loan_transaction_date', 'transaction_type':'loan_transaction_type', 'description':'loan_description'}, inplace=True)

transactionDetails.rename(columns={'id': 'transaction_id', 'created_at': 'transaction_created_at'}, inplace=True)

# Convert date columns to datetime format

merchantDetails[['merchant_created_at', 'merchant_updated_at']] = merchantDetails.apply(pd.to_datetime)

loanScheduleDetails[['loan_schedule_created_at', 'loan_schedule_updated_at', 'loan_schedule_paid_date', 'schedule_date']] = loanScheduleDetails.apply(pd.to_datetime)

loanLedgerDetails[['loan_created_at', 'loan_updated_at', 'loan_deleted_at']] = merchantDetails.apply(pd.to_datetime)

transactionDetails[['transaction_datetime', 'transaction_created_at']] = transactionDetails.apply(pd.to_datetime)

# Use loans only older than 3 months

# Calculate the cutoff date for loans older than 3 months
cutoff_date = pd.Timestamp.today() - pd.DateOffset(months=3)

loanSchedule = loanScheduleDetails[loanScheduleDetails['schedule_date'] < cutoff_date]

loanLedger = loanLedgerDetails[loanLedgerDetails['loan_created_at'] < cutoff_date]

# Merge the data on 'merchant_code' to get only merchants who have or have had loans

merchants = pd.merge(merchantDetails, loanSchedule[['merchant_code']], on='merchant_code', how='inner')

# Filter transactionDetails to only include merchants who have taken loans and transactions prior to loan disbursement

transactions = pd.merge(transactionDetails, loanSchedule[['merchant_code', 'loan_schedule_created_at']], on='merchant_code', how='inner')

transactions = transactions[transactions['transaction_created_at'] < transactions['loan_schedule_created_at']]

### Examining Data:

#### Columns and data types:

In [ ]:
merchants.head()
loanSchedule.head()
loanLedger.head()
transactions.head()



In [ ]:
merchants.info()
loanSchedule.info()
loanLedger.info()
transactions.info()

#### Dropping non-required columns:

In [ ]:
merchants=merchants.drop([''],axis=1)
loanSchedule=loanSchedule.drop([''],axis=1)
loanLedger=loanSchedule.drop([''],axis=1)
transactions=transactions.drop([''],axis=1)

#### Checking and handling missing values:

In [ ]:
missing_info_merchants = merchants.isnull().sum()
missing_info_loanSchedule = loanSchedule.isnull().sum()
missing_info_loanLedger = loanLedger.isnull().sum()
missing_info_transactions = transactions.isnull().sum()

### Feature engineering:

In [ ]:
df = pd.DataFrame()
df['merchant_code'] = merchants['merchant_code']

#### Merchant Characteristics

In [ ]:
# Filter loan ledger where transaction type is 'DISBURSEMENT'

disbursement_df = loanLedger[loanLedger['loan_transaction_type'] == 'DISBURSEMENT']

# Create a mapping from merchants to map merchant_created_at by merchant_code

merchant_creation_map = merchants.set_index('merchant_code')['merchant_created_at'].to_dict()

# Map merchant_created_at to disbursement_df based on merchant_code

disbursement_df['merchant_created_at'] = disbursement_df['merchant_code'].map(merchant_creation_map)

# Calculate the difference in days between merchant_created_at and transaction_date

disbursement_df['relationage'] = (disbursement_df['loan_transaction_date'] - disbursement_df['merchant_created_at']).dt.days

# Select only relevant columns: 'merchant_code' and 'age_at_disbursement'
disbursement_age_df = disbursement_df[['merchant_code', 'relationage']]

# Merge 'age_at_disbursement' into the existing DataFrame
df = pd.merge(df, disbursement_age_df, on='merchant_code', how='left')

#### Loan Characteristics

In [ ]:
# Group by 'merchant_code' and count unique 'pan' (customers)

unique_customers_df = transactions.groupby('merchant_code')['pan'].nunique().reset_index()

unique_customers_df.rename(columns={'pan': 'unique_customers'}, inplace=True)

df = pd.merge(df, unique_customers_df, on='merchant_code', how='left')

#### Transaction Characteristics

In [ ]:
def calculate_trans_char(transactions, sales_period_end=1, sales_period_start=28, suffix=''):

    # Merge disbursement date to transactions
    disbursement_dates_amounts = loanLedger[loanLedger['loan_transaction_type'] == 'DISBURSEMENT']

    transactions = pd.merge(transactions, disbursement_dates_amounts[['merchant_code', 'loan_transaction_date', 'debit']], on='merchant_code', how='left')
    results = []

    # Loop for each merchant
    for merchant_code in transactions['merchant_code'].unique():

        # Filter transactions for the specific merchant
        merchant_transactions = transactions[transactions['merchant_code'] == merchant_code]
        disbursement_date = merchant_transactions['loan_transaction_date'].iloc[0]

        # Filter transactions for the sales period before disbursement
        sales_period_transactions_all = merchant_transactions[
            (merchant_transactions['transaction_datetime'] >= (disbursement_date - pd.Timedelta(days=sales_period_start))) &
            (merchant_transactions['transaction_datetime'] <= (disbursement_date - pd.Timedelta(days=sales_period_end)))
        ]

        # Filter transactions by response_code = 0
        sales_period_transactions = sales_period_transactions_all[sales_period_transactions_all['response_code'] == 0]

       # Aggregate sales by date
        daily_sales = sales_period_transactions.groupby(sales_period_transactions['transaction_datetime'].dt.date)['amount'].sum().reset_index(name='daily_sales')
        daily_sales['transaction_datetime'] = pd.to_datetime(daily_sales['transaction_datetime'])

        # Ensure continuity of dates: fill in missing dates with 0 sales
        all_dates = pd.date_range(start=disbursement_date - pd.Timedelta(days=sales_period_start), end=disbursement_date - pd.Timedelta(days=sales_period_end))
        daily_sales_continuous = pd.DataFrame({'transaction_datetime': all_dates})
        daily_sales_continuous = pd.merge(daily_sales_continuous, daily_sales, on='transaction_datetime', how='left').fillna(0)
        
        # Average of total daily transaction value (in log)
        avg_daily_sales = daily_sales_continuous['amount'].mean()
        log_avg_transday = np.log1p(avg_daily_sales)
        
        # Average transaction sizes (in log)
        avg_transaction_size = sales_period_transactions['amount'].mean()
        log_avgtrans = np.log1p(avg_transaction_size)
        
        # Coefficient of variation of transaction sizes
        std_dev_transaction_size = sales_period_transactions['amount'].std()
        cv_trans = std_dev_transaction_size / avg_transaction_size if avg_transaction_size != 0 else 0
        
        # Coefficient of variation of total daily transaction values
        std_dev_daily_sales = daily_sales_continuous['amount'].std()
        cv_transday = std_dev_daily_sales / avg_daily_sales if avg_daily_sales != 0 else 0
        
        # Herfindahl-Hirschmann index of customers total transaction value
        customer_sales = sales_period_transactions.groupby('pan')['amount'].sum()
        total_sales = customer_sales.sum()
        customer_sales_proportion = customer_sales / total_sales
        HH_cust_trans = (customer_sales_proportion ** 2).sum()
        
        # Average of periods without any transactions (# of inactive days/total number of transaction days)
        transaction_dates = sales_period_transactions['transaction_datetime'].dt.date.unique()
        inactive_days = len(set(all_dates.date) - set(transaction_dates))
        total_transaction_days = len(transaction_dates)
        avg_inactivity = inactive_days / total_transaction_days if total_transaction_days != 0 else 0
        
        # Days since the period of longest inactivity
        all_dates_df = pd.DataFrame(all_dates, columns=['date'])
        all_dates_df['had_transaction'] = all_dates_df['date'].isin(transaction_dates)
        all_dates_df['inactive_period'] = (all_dates_df['had_transaction'] == False).astype(int).diff().ne(0).cumsum()
        inactive_periods = all_dates_df[all_dates_df['had_transaction'] == False].groupby('inactive_period')['date'].agg(['min', 'max', 'size'])
        longest_inactivity_period = inactive_periods.loc[inactive_periods['size'].idxmax()]
        days_since_max_inactivity = (disbursement_date.date() - longest_inactivity_period['max'].date()).days
        
        # Days since last transaction
        last_transaction_date = sales_period_transactions['transaction_datetime'].max().date()
        days_since_lasttrans = (disbursement_date.date() - last_transaction_date).days
        
        # (Relative to disbursal) Day with the largest transactions value
        largest_transaction_day = daily_sales.loc[daily_sales['daily_sales'].idxmax(), 'transaction_datetime']
        max_trans_dt = (largest_transaction_day - disbursement_date.date()).days
    
        # (Relative to disbursal) Day with most number of transactions
        daily_transaction_count = sales_period_transactions.groupby(sales_period_transactions['transaction_datetime'].dt.date).size().reset_index(name='transaction_count')
        most_transactions_day = daily_transaction_count.loc[daily_transaction_count['transaction_count'].idxmax(), 'transaction_datetime']
        max_transcount_dt = (most_transactions_day - disbursement_date.date()).days
        
        # Days since last transaction of most frequent customer at the time of disbursal.
        most_frequent_customer = sales_period_transactions.groupby('pan').size().idxmax()
        most_frequent_customer_transactions = sales_period_transactions[sales_period_transactions['pan'] == most_frequent_customer]
        most_frequent_customer_last_transaction_date = most_frequent_customer_transactions['transaction_datetime'].max().date()
        dayspast_freqcust = (disbursement_date.date() - most_frequent_customer_last_transaction_date).days
        
        # Days since last transaction of largest customer at the time of disbursal.
        largest_customer = sales_period_transactions.groupby('pan')['amount'].sum().idxmax()
        largest_customer_transactions = sales_period_transactions[sales_period_transactions['pan'] == largest_customer]
        last_transaction_largest_customer = largest_customer_transactions['transaction_datetime'].max().date()
        dayspast_largcust = (disbursement_date.date() - last_transaction_largest_customer).days
        
        # Number of distinct customers within period (in log).
        custcount = sales_period_transactions['pan'].nunique()
        log_custcount = np.log1p(custcount)
        
        # Share of total transaction value conducted on {DayOfWeek}
        sales_period_transactions['day_of_week'] = sales_period_transactions['transaction_datetime'].dt.dayofweek
        transaction_value_by_day = sales_period_transactions.groupby('day_of_week')['amount'].sum()
        total_transaction_value = sales_period_transactions['amount'].sum()
        share_by_day_of_week = transaction_value_by_day / total_transaction_value
        day_of_week_share = share_by_day_of_week.to_dict()
        
        shr_Mon_trans = day_of_week_share[0]
        shr_Tue_trans = day_of_week_share[1]
        shr_Wed_trans = day_of_week_share[2]
        shr_Thu_trans = day_of_week_share[3]
        shr_Fri_trans = day_of_week_share[4]
        shr_Sat_trans = day_of_week_share[5]
        shr_Sun_trans = day_of_week_share[6]
        
        results.append({'merchant_code': merchant_code, f'log_avg_transday{suffix}':log_avg_transday, f'log_avgtrans{suffix}':log_avgtrans, 
                        f'cv_trans{suffix}':cv_trans, f'cv_transday{suffix}':cv_transday, f'HH_cust_trans{suffix}':HH_cust_trans, 
                        f'avg_inactivity{suffix}':avg_inactivity, f'days_since_max_inactivity{suffix}':days_since_max_inactivity, 
                        f'days_since_lasttrans{suffix}':days_since_lasttrans, f'max_trans_dt{suffix}':max_trans_dt, 
                        f'max_transcount_dt{suffix}':max_transcount_dt, f'dayspast_freqcust{suffix}':dayspast_freqcust, 
                        f'dayspast_largcust{suffix}':dayspast_largcust, f'log_custcount{suffix}':log_custcount, 
                        f'shr_Mon_trans{suffix}':shr_Mon_trans, f'shr_Tue_trans{suffix}':shr_Tue_trans, 
                        f'shr_Wed_trans{suffix}':shr_Wed_trans, f'shr_Thu_trans{suffix}':shr_Thu_trans, 
                        f'shr_Fri_trans{suffix}':shr_Fri_trans, f'shr_Sat_trans{suffix}':shr_Sat_trans,
                        f'shr_Sun_trans{suffix}':shr_Sun_trans})

    trans_char_df = pd.DataFrame(results)



    return trans_char_df

# 91 day period (long term - lt)

trans_char_df_lt = calculate_trans_char(transactions, sales_period_end=1, sales_period_start=91, suffix='_lt')

# 91 - 64 (t_1) and 1 - 28 (t_2) periods to then calculate change over lt

trans_char_df_t_1 = calculate_trans_char(transactions, sales_period_end=64, sales_period_start=91, suffix='_t_1')

trans_char_df_t_2 = calculate_trans_char(transactions, sales_period_end=1, sales_period_start=28, suffix='_t_2')

## Notes: